# Flows and divergences

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.patches import Rectangle
from networkx.classes.digraph import DiGraph
from teaching_optimization.networks.draw_network import draw_network


# Definition of the the_network

Nodes:

In [ ]:
nodes = ['n_1', 'n_2', 'n_3', 'n_4', 'n_5', 'n_6', 'n_7']


We assign coordinates to nodes. This is used only for drawing.

In [ ]:
positions = {
    'n_1': (-1, 0),
    'n_2': (2, 3),
    'n_3': (2, -3),
    'n_4': (5, 3),
    'n_5': (5, -3),
    'n_6': (8, 3),
    'n_7': (8, -3),
}


Arcs

In [ ]:
arcs = [
    ('n_1', 'n_2', 2.3),
    ('n_1', 'n_3', 4),
    ('n_2', 'n_3', -1),
    ('n_3', 'n_5', 4.5),
    ('n_2', 'n_4', 3),
    ('n_4', 'n_2', -2.1),
    ('n_4', 'n_5', -5),
    ('n_5', 'n_4', -5),
    ('n_6', 'n_7', 2.5),
    ('n_7', 'n_6', 3),
]


# Network object
We create a directed graph:

In [ ]:
G: DiGraph = DiGraph()


We add nodes with position attributes

In [ ]:
for node in nodes:
    G.add_node(node, pos=positions[node])


We add arcs with weights

In [ ]:
for arc in arcs:
    G.add_edge(arc[0], arc[1], flow=arc[2])

edge_labels = nx.get_edge_attributes(G, name='flow')


We plot the the_network, as well as two gray areas defining a cut.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax = draw_network(
    the_network=G, attr_edge_labels='flow', ax=ax
)  # Add transparent gray rectangles covering specific nodes
gray_areas = [('n_1', 'n_2', 'n_3'), ('n_4', 'n_5', 'n_6', 'n_7')]

for i, nodes in enumerate(gray_areas):
    x_values = [positions[node][0] for node in nodes]
    y_values = [positions[node][1] for node in nodes]
    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    enlargement_factor = 1.2
    width = (max_x - min_x) * enlargement_factor
    height = (max_y - min_y) * enlargement_factor
    rect = Rectangle(
        (min_x - (width - (max_x - min_x)) / 2, min_y - (height - (max_y - min_y)) / 2),
        width,
        height,
        alpha=0.3,
        color='gray',
    )

    ax.add_patch(rect)
    if i == 0:
        ax.text(
            min_x - (width - (max_x - min_x)) / 2 + 0.1 * width,
            min_y - (height - (max_y - min_y)) / 2 + 0.9 * height,
            "M",
            fontsize=12,
            fontweight='bold',
            color='black',
        )


plt.show()


# Questions

Now, use the functionalities of the `networkx` package in general, and the `G` object in particular to answer
the following questions.

For each node, calculate the divergence.
Expected answers:

- `Divergence of node n_1 = 6.3`
- `Divergence of node n_2 = 1.8`
- `Divergence of node n_3 = 1.5`
- `Divergence of node n_4 = -5.1`
- `Divergence of node n_5 = -4.5`
- `Divergence of node n_6 = -0.5`
- `Divergence of node n_7 = 0.5`

In [ ]:
divergences = {}
for node in G.nodes():
    in_flow = sum(
        G[edge[0]][edge[1]]['flow'] for edge in G.in_edges(node)
    )
    out_flow = sum(
        G[edge[0]][edge[1]]['flow'] for edge in G.out_edges(node)
    )
    divergence = out_flow - in_flow
    divergences[node] = divergence
    print(f'Divergence of node {node} = {divergence:.2g}')



Identify if the node is a supply, demand or transit node.
Expected answers:

- `Type of node n_1: Supply`
- `Type of node n_2: Supply`
- `Type of node n_3: Supply`
- `Type of node n_4: Demand`
- `Type of node n_5: Demand`
- `Type of node n_6: Demand`
- `Type of node n_7: Supply`

In [ ]:
node_types = {}
for node, divergence in divergences.items():
    if divergence > 0:
        node_types[node] = 'Supply'
    elif divergence < 0:
        node_types[node] = 'Demand'
    else:
        node_types[node] = 'Transit'
    print(f'Type of node {node}: {node_types[node]}')


What is the total divergence in the the_network?
The total divergence is always zero.

In [ ]:
total_divergence = sum(divergences.values())
print(f'Total divergence = {total_divergence}')


Write the cut $\Gamma =(M, N \setminus M)$ defined by the gray areas.

In [ ]:
left_nodes = ('n_1', 'n_2', 'n_3')
right_nodes = ('n_4', 'n_5', 'n_6', 'n_7')


Identify the forward and backward arcs of the cut. Replace the ??? by the appropriate conditions

In [ ]:
forward_cut_edges = [
    edge
    for edge in G.edges()
    if edge[0] in left_nodes
    and edge[1] in right_nodes
]
backward_cut_edges = [
    edge
    for edge in G.edges()
    if edge[1] in left_nodes
    and edge[0] in right_nodes
]
print(f'Forward arcs of the cut: {forward_cut_edges}')
print(f'Backward arcs of the cut: {backward_cut_edges}')


Calculate the forward flow through that cut $\Gamma$.
Expected answer: `Forward flow: 7.5`

In [ ]:
forward_flow = sum(
    G[edge[0]][edge[1]]['flow'] for edge in forward_cut_edges
)
print(f'Forward flow: {forward_flow}')



Calculate the backward flow through that cut $\Gamma$.
Expected answer: `Backward flow: -2.1`

In [ ]:
backward_flow = sum(
    G[edge[0]][edge[1]]['flow'] for edge in backward_cut_edges
)
print(f'Backward flow: {backward_flow}')


Calculate the total flow through that cut $\Gamma$.
Expected answer: `Flow through the cut: 9.6`

In [ ]:
cut_flow = forward_flow - backward_flow
print(f'Flow through the cut: {cut_flow}')


Calculate the sum of the divergences of the nodes belonging to $\mathcal{M}$. What do you remark?

In [ ]:
sum_divergence_m = sum(
    divergences[node] for node in left_nodes
)
print(f'The sum of the divergences of the nodes belonging to M: {sum_divergence_m}')


Thus the flow through $\Gamma$ is equal to the sum of the divergences
of the nodes belonging to $\mathcal{M}$. This is always the case.

The intuition is the following. Consider an arc $(i,j)$ such that both
$i$ and $j$ are in $\mathcal{M}$.
When calculating the sum of the
divergences in $\mathcal{M}$, the flow of this arc will be involved once
with a positive sign for the divergence of $j$, and once with a
negative sign for the divergence of $i$. Therefore, its contribution
to the sum is zero. It means that only arcs $(i,j)$ such that $i \in
\mathcal{M}$ and $j \in \mathcal{N} \setminus \mathcal{M}$ will
contribute to the sum. But these are exactly the arcs belonging to the
cut. Therefore,
$$ \sum_{i \in \mathcal{M}} \text{div}(x)_i = \sum_{(i,j) \in \Gamma^\rightarrow} x_{ij} - \sum_{(i,j) \in
\Gamma^\leftarrow} x_{ij} =  X (\Gamma).$$